In [2]:
from collections import Counter
from math import floor

import numpy as np

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Activation

Using TensorFlow backend.


In [3]:
with open('wikitext-2/wiki.train.tokens') as f:
    data = f.read()
data = data.replace('\n', '<eos>') #suggested preprocessing from source
print(data[:500])

 <eos> = Valkyria Chronicles III = <eos> <eos> Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story 


In [4]:
split = data.split()
tokens = Counter(split)
print(tokens.most_common()[:5])

VOCAB_SIZE = len(tokens)
print(f'\n Number of unique tokens: {VOCAB_SIZE}')

[('the', 113161), (',', 99913), ('.', 73388), ('of', 56889), ('<unk>', 54625)]

 Number of unique tokens: 33278


In [5]:
stoi, itos = {}, {}
for i, (tok, _) in enumerate(tokens.items()):
    stoi[tok] = i; itos[i] = tok
#end for

print(itos[0])
print(stoi['<eos>'])

<eos>
0


Now we have integer representations of all unique tokens in the dataset. (This could have been done with the native [kears tokenizer](https://keras.io/preprocessing/text/#tokenizer) , but since the dataset was already cleaned for us, I guess we could still achive similar results). The most simple approach to turn these into vectors is to use the integers as a one hot encoding paramter. This approch is however quite poor since all words in this embedding-space are equidistant. A better approach that I will try later is to use a dence representation, which we will include in the training of our network, where each word share features with eachother.

In [6]:
oh_data = to_categorical([stoi[tok] for tok in split])

In [7]:
print(len(oh_data[0])) #one-hot representation
print(len(oh_data))

33278
2088628


In [8]:
seq_len = 10
n_seq = floor(len(oh_data)/seq_len)
n_seq = 20
em_sz = len(oh_data[0])

#Create each training sequence
X_train, Y_train = np.zeros((n_seq, seq_len, em_sz)), np.zeros((n_seq, seq_len, em_sz))
for i in range(n_seq):
    X_train[i,:,:] = oh_data[i*seq_len:(i+1)*seq_len]
    Y_train[i,:,:] = oh_data[i*seq_len+1:(i+1)*seq_len+1]

To simulate randomly shuffeling our training data, we cannot change the order of our token, but we can change the length or input sequences. This is something that we need to implement later. 

Now its time to feed this data into a model. The goal here is to predict the next word given a set of previous ones. 

In [9]:
h_dim = 10
n_layer = 2

model = Sequential()
model.add(LSTM(h_dim, input_shape=(None, em_sz), return_sequences=True))
for i in range(n_layer - 1):
    model.add(LSTM(h_dim, return_sequences=True))
model.add(TimeDistributed(Dense(em_sz)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [ ]:
nb_epoch = 0
bs = 2
while True:
    print('\n\n')
    model.fit(X_train, Y_train, batch_size=bs, verbose=1, epochs=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH)
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))




Epoch 1/1
